### Fase 1: Extracción de Datos de API de MoviesDataset

En esta fase se utilizará la API de MoviesDataset para extraer información relevante sobre películas y cortometrajes.

**Requerimientos:**

- Películas desde 1990 hasta la actualidad.
- Géneros: Drama, Comedy, Action, Fantasy, Horror, Mystery, Romance, Thriller.
- Información necesaria:
    - Tipo (corto o película).
    - Nombre.
    - Año y mes de estreno.
    - ID de la película.

**Nota:** Los datos extraídos deberán almacenarse en una lista de tuplas.

In [197]:
import requests
import pandas as pd
from tqdm import tqdm

import pickle 
import sys
sys.path.append('../')

Tipos

In [198]:
def get_types():

	url = "https://moviesdatabase.p.rapidapi.com/titles/utils/titleTypes"

	headers = {
		"x-rapidapi-key": "8b197213b8mshd78a73a5116ae9fp1f4320jsne0ab638c2fe8",
		"x-rapidapi-host": "moviesdatabase.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers)

	return response.json()['results']

In [199]:
get_types()

[None,
 'movie',
 'musicVideo',
 'podcastEpisode',
 'podcastSeries',
 'short',
 'tvEpisode',
 'tvMiniSeries',
 'tvMovie',
 'tvPilot',
 'tvSeries',
 'tvShort',
 'tvSpecial',
 'video',
 'videoGame']

Tenemos 'movie' y 'short' como puntos de interés

Comprobamos que los géneros son correctos

In [200]:
def get_genres():

	url = "https://moviesdatabase.p.rapidapi.com/titles/utils/genres"

	headers = {
		"x-rapidapi-key": "8b197213b8mshd78a73a5116ae9fp1f4320jsne0ab638c2fe8",
		"x-rapidapi-host": "moviesdatabase.p.rapidapi.com"
	}

	response = requests.get(url, headers=headers)

	return response.json()['results']

In [201]:
list_genres = get_genres()

In [202]:
genres = ['Drama', 'Comedy', 'Action', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Thriller']

for genre in genres:
    print(genre, genre in list_genres)

Drama True
Comedy True
Action True
Fantasy True
Horror True
Mystery True
Romance True
Thriller True


---

In [203]:
def api_call(genre, page, titleType):

    # Lista vacía para almacenar los resultados
    data = []

    # Endpoint
    url = f"https://moviesdatabase.p.rapidapi.com/titles?endYear=2024&genre={genre}&page={page}&startYear=1990&titleType={titleType}"

    # Pasamos los headers
    headers = {
        "x-rapidapi-key": "8b197213b8mshd78a73a5116ae9fp1f4320jsne0ab638c2fe8",
        "x-rapidapi-host": "moviesdatabase.p.rapidapi.com"
    }

    # Respuesta
    response = requests.get(url, headers=headers)

    # Si hay algún error devolvemos
    if response.status_code != 200:
        print("Error:", response.status_code)
        return [data]
    
    # Si ya hemos alcanzado el máximo de páginas salimos
    if page != response.json()['page']:
        print(f"{genre} and {titleType} finished at page {page}")
        return [data]
    
    # Obtenemos los resultados
    results = response.json()['results']

    # Recuperamos la información de cada película
    for i in range(len(results)):

        type_ = titleType
        title_ = results[i]['titleText']['text']

        try:
            year_ = results[i]['releaseDate']['year']
            month_ = results[i]['releaseDate']['month']
        except:
            year_ = None
            month_ = None

        id_ = results[i]['id']
        genre_ = genre

        # Añadimos la infromación a la lista
        data.append((type_, title_, year_, month_, id_, genre_))
    
    return data, results

In [204]:
# Prueba
genre = 'Drama'
titleType = 'movie'
page = str(1)

api_call(genre, page, titleType)

([('movie', "Mom's Coming", 2024, 11, 'tt10579986', 'Drama'),
  ('movie', 'Love Song and Power', 2024, 1, 'tt10597228', 'Drama'),
  ('movie', 'Spaceman', 2024, 3, 'tt11097384', 'Drama'),
  ('movie', 'Amigos imaginarios', 2024, 5, 'tt11152168', 'Drama'),
  ('movie', 'Manje Bistre 3', 2024, 7, 'tt11822244', 'Drama'),
  ('movie', 'Yi dao tian tang', 2024, None, 'tt12861438', 'Drama'),
  ('movie', 'Broken Glass', 2024, 2, 'tt13036100', 'Drama'),
  ('movie', 'Fără zgomot: Ziua 1', 2024, 6, 'tt13433802', 'Drama'),
  ('movie', 'Three Couples Freed', 2024, 1, 'tt13561128', 'Drama'),
  ('movie', 'Fighter', 2024, 1, 'tt13818368', 'Drama')],
 [{'_id': '61e58ea278c012aa34446beb',
   'id': 'tt10579986',
   'primaryImage': {'id': 'rm878544897',
    'width': 1217,
    'height': 1792,
    'url': 'https://m.media-amazon.com/images/M/MV5BYjgyN2RjZWEtYzRjYy00ZGEyLWI4MTItNjk5OGI1YThhZWNlXkEyXkFqcGc@._V1_.jpg',
    'caption': {'plainText': "Mom's Coming (2024)", '__typename': 'Markdown'},
    '__typename':

In [205]:
genres

['Drama',
 'Comedy',
 'Action',
 'Fantasy',
 'Horror',
 'Mystery',
 'Romance',
 'Thriller']

Almacenamos género a género por si nos da algún problema tener todos separados y no tener que repetirlos

In [224]:
# Recorremos cada género
for genre in genres:
    break # Para evitar ejecutar esto sin querer

    # Donde almacenaremos todos los datos
    datos = []

    # Recorremos todos los tipos
    for titleType in ['movie', 'short']:

        # Iteramos por página
        for i in tqdm(range(1, 101)):
            datos_llamada = api_call(genre, str(i), titleType)[0]

            if datos_llamada == []:
                print("Step finished")
                break

            datos.extend(datos_llamada)

    with open(f'../data/datos_{genre.lower()}.pkl', 'wb') as f:
        pickle.dump(datos, f)

 50%|█████     | 50/100 [00:13<00:13,  3.75it/s]


Thriller and movie finished at page 51
Step finished


 50%|█████     | 50/100 [00:13<00:13,  3.72it/s]

Thriller and short finished at page 51
Step finished


Vamos a juntar todo

In [234]:
full_data = []

for genre in genres:
    print(genre)

    with open(f'../data/datos_{genre.lower()}.pkl', 'rb') as f:
        single_data = pickle.load(f)

    full_data.extend(single_data)

# Guardamos todo
with open(f'../data/full_data.pkl', 'wb') as f:
    pickle.dump(full_data, f)

Drama
Comedy
Action
Fantasy
Horror
Mystery
Romance
Thriller


In [235]:
len(full_data)

8000

Tenemos un total de 8000 entradas

In [236]:
full_data

[('movie', "Mom's Coming", 2024, 11, 'tt10579986', 'Drama'),
 ('movie', 'Love Song and Power', 2024, 1, 'tt10597228', 'Drama'),
 ('movie', 'Spaceman', 2024, 3, 'tt11097384', 'Drama'),
 ('movie', 'Amigos imaginarios', 2024, 5, 'tt11152168', 'Drama'),
 ('movie', 'Manje Bistre 3', 2024, 7, 'tt11822244', 'Drama'),
 ('movie', 'Yi dao tian tang', 2024, None, 'tt12861438', 'Drama'),
 ('movie', 'Broken Glass', 2024, 2, 'tt13036100', 'Drama'),
 ('movie', 'Fără zgomot: Ziua 1', 2024, 6, 'tt13433802', 'Drama'),
 ('movie', 'Three Couples Freed', 2024, 1, 'tt13561128', 'Drama'),
 ('movie', 'Fighter', 2024, 1, 'tt13818368', 'Drama'),
 ('movie', 'An Irish Angel', 2024, 4, 'tt14117392', 'Drama'),
 ('movie', 'MoviePass, MovieCrash', 2024, 5, 'tt14257662', 'Drama'),
 ('movie', 'Rhapsody in Justice', 2024, 8, 'tt14478740', 'Drama'),
 ('movie', 'Guntur Kaaram', 2024, 1, 'tt14564000', 'Drama'),
 ('movie', 'Noda Hitam Dalam Pelukan', 2024, 11, 'tt14643262', 'Drama'),
 ('movie', 'La Puerta Verde', 2024, 8, '